In [1]:
import os, glob, platform
import numpy as np
import numpy.matlib
import pickle
import pandas as pd
import pathlib
import matplotlib
import matplotlib.pyplot as plt
import mne
mne.__version__
from mne.viz import plot_alignment, snapshot_brain_montage
import shutil
from mne.datasets import eegbci
from sklearn.model_selection import train_test_split

# from mne_bids import write_raw_bids, BIDSPath, print_dir_tree, make_dataset_description
# from mne_bids.stats import count_events
import sys


In [2]:
path_utils = '/decoding_toolbox_py/helper_funcs' 
sys.path.append(path_utils)

In [3]:
''' VARIABLES '''

dataset = 'eeg'

amount_of_subjects = 2 # Change the range so the process is faster
if amount_of_subjects > 26: amount_of_subjects = 26
subjs_list = ['s{:02d}'.format(i) for i in range(1, amount_of_subjects+1) if i != 6 ] 
print(subjs_list)
nSubj = len(subjs_list)

numC = 8

angles = [i * 180./numC for i in range(numC)]

x_labels = np.array(angles)

resample = True # speeds up the procees but showing worse results overall
if resample: resample_frequency = 20 # in Hz, original freq is 500Hz

cfg_stim = dict()
cfg_stim['kappa'] = 4
cfg_stim['NumC'] = numC
cfg_stim['Tuning'] = 'vonmises'
# cfg_stim['Tuning'] = 'halfRectCos'
cfg_stim['offset'] = 0

cfg_train = dict()
cfg_train['gamma'] = 0.1
cfg_train['demean'] = True
cfg_train['returnPattern'] = True

cfg_test = dict()
cfg_test['demean'] = 'traindata'

['s01', 's02']


In [4]:
'''EEG Dataset'''
def read_data(
        number_of_repetition=3,
        resample=False,
        subjs_list = subjs_list,
        task = 'stim'
        ):
    path = 'Cond_CJ_EEG'

    epochs = []
    all_epochs = []
    all_rawdata = []
    all_st_epochs = []
    all_st_rawdata = []
    for subject_id in subjs_list:
        preproc_path = os.path.join(path, subject_id)

        if task == 'main':
            epoch = mne.read_epochs(os.path.join(preproc_path, 'main_epo.fif'), verbose=False)
            epochs.append(epoch.average())
            all_epochs.append(epoch)
            all_rawdata.append({'epoch_dat': epoch.get_data(), 'metadata': epoch.metadata})
            
        if task == 'stim':
        
            st_epoch = mne.read_epochs(os.path.join(preproc_path, 'mainstim_epo.fif'), verbose=False)
            # print(st_epoch.info['sfreq'])
            if resample: 
                print('Frequency before:', st_epoch.info['sfreq'])
                st_epoch = st_epoch.resample(resample_frequency)
                print('Frequency after:' ,st_epoch.info['sfreq'])
                
            all_st_epochs.append(st_epoch)
            all_st_rawdata.append(
                {
                'epoch_dat': st_epoch.get_data()[st_epoch.metadata['nrep'] == number_of_repetition,:,:] ,
                'metadata': st_epoch.metadata[st_epoch.metadata['nrep'] == number_of_repetition]
                }
                )
    if task == 'main':
        return all_rawdata
    else:
        return all_st_rawdata


In [5]:
all_rawdata = read_data (task = 'main')

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


In [6]:
all_rawdata[0]['metadata'].columns

Index(['index', 'subj', 'nblock', 'ntrial', 'nrep', 'trial_type', 'cond-1',
       'cond', 'rDV', 'DV', 'resp', 'deci-2', 'deci-1', 'deci', 'corr-1',
       'r_map', 'correct', 'confi', 'RT', 'd1', 'conf_lvl', 'correct-1', 'd2',
       'd3', 'd4', 'd5', 'd6', 'o1', 'o2', 'o3', 'o4', 'o5', 'o6', 'confi-1',
       'conf_lvl-1'],
      dtype='object')

In [57]:

A = np.random.random((3,2,5))
print(A)
A = A.reshape((3,-1))
print(A.shape)
A

[[[0.76667744 0.64466872 0.17268001 0.3703947  0.31607555]
  [0.82125675 0.87009374 0.48973327 0.05711375 0.87082003]]

 [[0.45332208 0.22615084 0.05847094 0.07080563 0.36377101]
  [0.186528   0.00463693 0.33529402 0.78936992 0.41939425]]

 [[0.92074345 0.0731833  0.49209385 0.94387508 0.52089158]
  [0.21512982 0.89986887 0.55144138 0.36536059 0.2365453 ]]]
(3, 10)


array([[0.76667744, 0.64466872, 0.17268001, 0.3703947 , 0.31607555,
        0.82125675, 0.87009374, 0.48973327, 0.05711375, 0.87082003],
       [0.45332208, 0.22615084, 0.05847094, 0.07080563, 0.36377101,
        0.186528  , 0.00463693, 0.33529402, 0.78936992, 0.41939425],
       [0.92074345, 0.0731833 , 0.49209385, 0.94387508, 0.52089158,
        0.21512982, 0.89986887, 0.55144138, 0.36536059, 0.2365453 ]])

In [56]:

nSubj = 1
add_repetitions = True
add_labels = True
X = np.vstack([all_rawdata[i]['epoch_dat'] for i in range(nSubj)])
print(X.shape)
X = X.reshape((X.shape[0], -1))
y = np.concatenate([all_rawdata[i]['metadata']['deci'] for i in range(nSubj)])
print(X.shape, y.shape)
if add_repetitions:
    for i in range (nSubj):
        nrep = np.array(all_rawdata[i]['metadata']['nrep'])
        nrep = nrep.reshape(-1,1)
        X = np.hstack((X, nrep))
if add_labels:
    pass

print(X.shape, y.shape)

(250, 32, 2876)
(250, 92032) (250,)
(250, 92033) (250,)


In [46]:
X = all_rawdata[0]['epoch_dat']
X = X.reshape((250, -1))
y = all_rawdata[0]['metadata']['deci']
nrep = np.array(all_rawdata[0]['metadata']['nrep'])
print(X.shape, nrep.shape)
nrep = nrep.reshape(-1, 1)
X = np.hstack((X, nrep))
# X = all_rawdata[0]['epoch_dat']
# y = all_rawdata[0]['metadata']['deci']
# for i in range(1, nSubj):
#     X = np.concatenate((X, all_rawdata[i]['epoch_dat']))
#     y = np.concatenate((y, all_rawdata[i]['metadata']['deci'] ))
# print(X.shape, y.shape)
# X = X.reshape(6205,-1)
# print(X.shape)
print(X.shape)

(250, 92032) (250,)
(250, 92033)


In [47]:
from sklearn.svm import SVC
# from pyrcn.echo_state_network import ESNClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train.shape, y_train.shape)

clf = AdaBoostClassifier()

clf = GradientBoostingClassifier()

clf = LogisticRegression()

clf = SVC()
clf = CatBoostClassifier()

clf = LogisticRegression()

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy on test set:", accuracy)



(200, 92033) (200,)
Accuracy on test set: 0.7


RF all subjects 80-20 = 0.556809024979855

RF, Logistic, SVC = 1 sub 0.66

XGB 1 sub 0.62

XGB 26 0.5495568090249798

Ada 0.6

GB 0.62

0.66
